In [ ]:
# 1️⃣ Vérifier le GPU
!nvidia-smi

In [ ]:
# 2️⃣ Installer les dépendances
%pip install ultralytics opencv-python scikit-image

In [ ]:
# 3️⃣ Monter Google Drive (pour sauvegarder le modèle)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 4️⃣ Créer la structure des dossiers
import os

os.makedirs('datasets/train/images', exist_ok=True)
os.makedirs('datasets/train/labels', exist_ok=True)
os.makedirs('datasets/val/images', exist_ok=True)
os.makedirs('datasets/val/labels', exist_ok=True)

print("Structure créée!")
print("Uploadez vos images dans datasets/train/images et datasets/val/images")
print("Uploadez vos labels dans datasets/train/labels et datasets/val/labels")

In [ ]:
# 5️⃣ OPTION A: Uploader un ZIP avec vos données
# Décommentez et exécutez si vous avez un fichier ZIP

# from google.colab import files
# uploaded = files.upload()  # Sélectionnez votre datasets.zip
# !unzip datasets.zip -d datasets/

In [ ]:
# 5️⃣ OPTION B: Copier depuis Google Drive
# Décommentez si vos données sont sur Drive

# !cp -r "/content/drive/MyDrive/datasets/train" datasets/
# !cp -r "/content/drive/MyDrive/datasets/val" datasets/

In [ ]:
# 6️⃣ Créer le fichier data.yaml
data_yaml = """
train: datasets/train/images
val: datasets/val/images

nc: 15
names:
  - cat1
  - cat2
  - cat3
  - cat4
  - cat5
  - cat6
  - cat7
  - cat8
  - cat9
  - cat10
  - cat11
  - cat12
  - cat13
  - cat14
  - cat15
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml)

print("✅ data.yaml créé!")
print("⚠️ N'oubliez pas de modifier les noms des catégories!")

In [ ]:
# 7️⃣ Vérifier les données
import os

train_images = len(os.listdir('datasets/train/images')) if os.path.exists('datasets/train/images') else 0
train_labels = len(os.listdir('datasets/train/labels')) if os.path.exists('datasets/train/labels') else 0
val_images = len(os.listdir('datasets/val/images')) if os.path.exists('datasets/val/images') else 0
val_labels = len(os.listdir('datasets/val/labels')) if os.path.exists('datasets/val/labels') else 0

print(f"📊 Train: {train_images} images, {train_labels} labels")
print(f"📊 Val: {val_images} images, {val_labels} labels")

if train_images == 0:
    print("\n⚠️ Aucune image trouvée! Uploadez vos données d'abord.")

In [ ]:
# 8️⃣ ENTRAÎNEMENT YOLOv8 🚀
from ultralytics import YOLO

# Charger le modèle pré-entraîné
model = YOLO('yolov8n.pt')

# Entraînement (optimisé pour Colab GPU T4)
results = model.train(
    data='data.yaml',
    epochs=20,
    imgsz=640,          # Colab peut gérer 640!
    batch=16,           # Batch plus grand = plus rapide
    device=0,           # GPU
    workers=2,
    amp=True,           # Mixed precision
    cache=True,
    name='yolo15cats',
    patience=5,         # Early stopping
    verbose=True
)

print("\n✅ Entraînement terminé!")

In [ ]:
# Évaluation du modèle
metrics = model.val()

print("\nRésultats d'évaluation:")
print(f"   mAP50: {metrics.box.map50:.3f}")
print(f"   mAP50-95: {metrics.box.map:.3f}")
print(f"   Precision: {metrics.box.mp:.3f}")
print(f"   Recall: {metrics.box.mr:.3f}")

In [ ]:
# Sauvegarder le modèle sur Google Drive
import shutil

# Créer le dossier de destination
os.makedirs('/content/drive/MyDrive/ml-extraction-project/models', exist_ok=True)

# Copier le meilleur modèle
shutil.copy(
    'runs/detect/yolo15cats/weights/best.pt',
    '/content/drive/MyDrive/ml-extraction-project/models/yolo15cats.pt'
)

print("Modèle sauvegardé sur Google Drive!")
print("Chemin: /MyDrive/ml-extraction-project/models/yolo15cats.pt")

In [ ]:
# Télécharger le modèle directement
from google.colab import files

files.download('runs/detect/yolo15cats/weights/best.pt')
print("\nTéléchargement du modèle lancé!")
print("Placez-le dans: backend/data/models/yolo15cats.pt")

In [ ]:
# 1️⃣2️⃣ Test rapide sur une image
from PIL import Image
import matplotlib.pyplot as plt

# Charger le meilleur modèle
best_model = YOLO('runs/detect/yolo15cats/weights/best.pt')

# Tester sur une image de validation
val_images = os.listdir('datasets/val/images')
if val_images:
    test_img = f'datasets/val/images/{val_images[0]}'
    results = best_model.predict(test_img, save=True)
    
    # Afficher le résultat
    result_img = Image.open(f'runs/detect/predict/{val_images[0]}')
    plt.figure(figsize=(10, 10))
    plt.imshow(result_img)
    plt.axis('off')
    plt.title('Test de détection')
    plt.show()
else:
    print("⚠️ Pas d'images de validation pour tester")